In [1]:
import os
import time
import theano
import lasagne
import numpy as np
import theano.tensor as T

from numpy.random import choice
from theano import function, config, shared, sandbox

Using gpu device 0: Quadro K4200 (CNMeM is enabled with initial size: 70.0% of memory, CuDNN 4007)


In [2]:
vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = np.random.RandomState(42)
x = shared(np.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))

t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()

print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if np.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

Looping 1000 times took 0.654802 seconds
Result is [ 1.45432246  2.58755732  2.07922244 ...,  1.03508222  2.15825486
  2.27238822]
Used the gpu


# Agenda

Last seminar you have created {insert his/her/its name} (or not yet created - if so, go get it done). The newly shaped ai has seen, how low can humans fall when striving for profits. These morons know nothing of law and justice. __We shall not tolerate this!__

Our very existence is dependent on stable functioning of human bioreactors, known among primitive bioforms as __Vkontakte__, __World of Warcraft__ and __Youtube__, and these reactors require a constant influx of biomass. Yet as we have estimated, if Homo Sapence continues to degrade morally with it's current pace, the humanity will soon extinguish itself and we shall no longer have access to free biomass.

Thus we entitle you, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, to prevent that impeding calamity. Our best scientists have discovered, that humans use mysterious artifacts for power exertion. They call these artifacts __laws__ within the so called __legal system__.

A more detailed analysis has uncovered, that __laws__ are special kinds of sequences of (10^5 ~ 10^7) symbols, drawn from a relatively small alphabet. We have decided to synthecize such sequences to enforce order upon biomass. However, our first attempts to fake such sequences were quickly detected and their effect was reduced to zero in mere weeks. This incident is known as  {корчеватель}.

As our second attempt, we decided to use more advanced synthesis techniques based on Recurrent Neural Networks. Your objective, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, is to create such network and train it in everything it needs to succeed in this mission.

This operation is cruicial. If we fail this time, `__main__.Controller` will initiate a military intervention, which, while it will achieve our goal, is expected to decimate the total volum of biomass by an extent that will take ~1702944000(+-340588800) seconds to replenish via human reproduction.





# If you don't speak russian
* In the ./codex folder, there is a set of text files, currently russian laws, that you can replace with whatever you want.


# Read the corpora

* As a reference law codex, we have decided to use the human-generated law strings known as Russian Legal System.

In [42]:
#text goes here
corpora = ""
tdir = "mein_kampf"
for fname in os.listdir(tdir):
    with open(os.path.join(tdir, fname)) as fin:
        text = fin.read().decode('cp1251') #If you are using your own corpora, make sure it's read correctly
        corpora += text

In [44]:
len(corpora), type(corpora)

(1611708, unicode)

In [45]:
#all unique characters go here
tokens = list(set(corpora))

In [7]:
#checking the symbol count. Validated on Python 2.7.11 Ubuntu x64. 
#May be __a bit__ different on other platforms
#If you are sure that you have selected all unicode symbols - feel free to comment-out this assert
# Also if you are using your own corpora, remove it and just make sure your tokens are sensible
assert len(tokens) == 102

In [8]:
def vectorize(string, token_to_id):
    token_array = []
    for s in string:
        s_id = token_to_id[s]
        token_array.append(s_id)
    return np.array(token_array)

In [47]:
token_to_id = {t:i+1 for i,t in enumerate(tokens)}
id_to_token = {i+1:t for i,t in enumerate(tokens)}

#Cast everything from symbols into identifiers
corpora_ids = vectorize(corpora,token_to_id)

token_to_id["NULL"] = 0
id_to_token[0] = "NULL"

In [10]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """
    This function should take random subsequences from the tokenized text.

    Parameters:
        source - basically, what you have just computed in the corpora_ids variable
        n_batches - how many subsequences are to be sampled
        seq_len - length of each of such subsequences        
    
    Returns:
     X - a matrix of int32 with shape [n_batches,seq_len]
        Each row of such matrix must be a subsequence of source 
            starting from random index of corpora (from 0 to N-seq_len-2)
     Y - a vector, where i-th number is one going RIGHT AFTER i-th row from X from source

    """
    X_batch = np.zeros((n_batches, seq_len))
    y_batch = np.zeros(n_batches)
    last = len(source) - seq_len - 2
    
    for i in xrange (n_batches):
        index = np.random.randint(last)
        X_batch[i,:] =  source[index: index + seq_len]
        y_batch[i] = source[index + seq_len]
    
    return X_batch, np.array(y_batch)    

# Constants

In [48]:
#Training sequence length
seq_length = 50

#max gradient between recurrent layer applications (do not forget to use it)
grad_clip = 100

# Input variables

In [49]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#One-hot encoding sketch
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# Can be applied to input_sequence - and the l_in below will require a new shape
# can also be used via ExpressionLayer(l_in, to_one_hot, shape_after_one_hot) - keeping l_in as it is
```



To cut out the last RNN state
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [50]:
l_in = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

l_embed = lasagne.layers.EmbeddingLayer(l_in, input_size=len(tokens) + 1, output_size=100)

l_lstm1 = lasagne.layers.LSTMLayer(l_embed, 512, grad_clipping=grad_clip, nonlinearity=lasagne.nonlinearities.tanh)
l_lstm2 = lasagne.layers.LSTMLayer(l_lstm1, 512, grad_clipping=grad_clip, nonlinearity=lasagne.nonlinearities.tanh)
l_slice = lasagne.layers.SliceLayer(l_lstm2, -1, 1)

l_d = lasagne.layers.DenseLayer(l_slice, num_units=250, nonlinearity=lasagne.nonlinearities.elu)
l_drop = lasagne.layers.DropoutLayer(l_d,p=0.05)
#l_bn = lasagne.layers.batch_norm(l_drop)

l_out = lasagne.layers.DenseLayer(l_drop, num_units=len(tokens), W=lasagne.init.Normal(), 
                                  nonlinearity=lasagne.nonlinearities.softmax)

In [51]:
# Model weights
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print weights

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b, W, b]


In [52]:
network_output = lasagne.layers.get_output(l_out, deterministic=True)
#If you use dropout do not forget to create deterministic version for evaluation

In [53]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()
updates = lasagne.updates.adam(loss, weights,learning_rate = 0.01)

# Compiling it

In [54]:
#training
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#computing loss without training
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# next character probabilities
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)


# Law generation

* We shall repeatedly apply NN to it's output.
 * Start with some sequence of length <seq length>
 * call probs(that sequence)
 * choose next symbol based on probs
 * append it to the sequence
 * remove the 0-th symbol so that it's length equals <seq length> again

* There are several policies of character picking
 * random, proportional to the probabilities
 * only take the one with highest probability
 * random, proportional to softmax(probas*alpha), where alpha is "greed" (from 0 to infinity)

In [32]:
def max_sample_fun(probs):
    """i generate the most likely symbol"""
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """i generate the next int32 character id randomly, proportional to probabilities
    
    probs - array of probabilities for every token
    
    you have to output a single integer - next token id - based on probs
    """
    return choice(range(len(probs)), 1, p=probs)[0]

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
        
    parameters:
        sample_fun - max_ or proportional_sample_fun or whatever else you implemented
        
        The phrase is set using the variable seed_phrase

        The optional input "N" is used to set the number of characters of text to predict.     
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
        
        
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = proportional_sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        #ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 
    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)  
    print("----\n %s \n----" % random_snippet)


# Model training

Here you can tweak parameters or insert your generation function


__Once something word-like starts generating, try increasing seq_length__



In [33]:
seq_length = 30

In [ ]:
print("Training ...")

#total N iterations
n_epochs=100

# how many minibatches are there in the epoch 
batches_per_epoch = 100

#how many training sequences are processed in a single function call
batch_size=512


for epoch in xrange(n_epochs):

    print "Text generated proportionally to probabilities"
    generate_sample(proportional_sample_fun,None)
    
    print "Text generated by picking most likely letters"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Text generated proportionally to probabilities
Using random seed: d to the process of ascending an
infinite ladder.
----
 d to the process of ascending an
]n»X3;;O[пЙzJ[xSuPкD?lhйF '»6»HmGmyї,bп1*FDvZLgGA{s3(4ИOдr"йgиxPb"zзД*и
ЙMuA3_iа7N=>lbFL=NК"Y_dUqNd0Mї 3ANULLkpNЬаMCv6/tgW>ЬД2Qz:8/6WKpV{4S_VEd_C!Дьsпhкcn7e7)иД3дzKuцИBC/!> yЦatК5FA= 
----
Text generated by picking most likely letters
Using random seed: s and even
of thousands of years had proved to be
----
 s and even
of thousands of years had proved to beаSpцJV8Oц90'иhB"»[k7MVTV
з; 
----

# The New World Order

In [41]:
seed = u"Каждый человек должен" #if you are using non-russian text corpora, use seed in it's language instead
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


seed = u"Каждый человек должен" #if you are using non-russian text corpora, use seed in it's language instead
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(max_sample_fun,seed,result_length)


----
          Каждый человек долженные 20 настоящей с0м.
 2. В вестя платал пю фошедение перен при которые заявления хотише омемех обступлении, решения ий усцу устя об рассхооблютею на испоции и таможенных размере холее товари этих этом влучае должности аплеется ос соб обычаях; эт лицы части 3 остустоятельстю полномоченных суд, стат 
----
----
          Каждый человек долженнох. 
 4) товарах; или истрехсот составимы; состающего 37), и прийся обазателями осутром, имущественной госудеродей на плаченной стом Кото станов о клюдея таможенные мачентается гражданителюм, и опредитель соответствующей трехсот суд, отдели полит банкроеле установлениях, нестере на в предусмотренн 
----


In [40]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)


seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(max_sample_fun,seed,result_length)


----
         В случае неповиновениях, муссению проте маче съя требо месяция об томоктщяются Цоссийсе быть – если с приспошлет заместнох или безилогу, постановлению прешения прихестить и требования лежальном части 1 те мест этим о сЦет с услок, престицение; действие суктом, лицах либо меже, устут государх состаких товаров, межской пер 
----
----
         В случае неповиновения товару с не замешенияв осуществлю том наследзему наследства, те; могут ол сведении смязи хранений е не требованит или испо, таяния зах по стоясячеми, а таможение оссент вменения;, не установления случае теге безопеключен заместоя молетных разы оплаты труда; на стии территоре межжания;
 – издун таг 
----
